#Edge Detection

In this episode, we will learn how to use `skimage` functions to apply *edge detection* to an image. In edge detection, we find the boundaries or edges of objects in an image, by determining where the brightness of the image changes dramatically. Edge detection can be used to extract the structure of objects in an image. If we are interested in the number, size, shape, or relative location of objects in an image, edge detection allows us to focus on the parts of the image most helpful, while ignoring parts of the image that will not help us.

For example, once we have found the edges of the objects in the image (or once we have converted the image to binary using thresholding), we can use that information to find the *connected components* of an image, which we will learn about in the following episode. With these, we can do things like count the number of objects in the image, measure the size of the objects, classify the shapes of the objects, and so on.

As was the case for blurring and thresholding, there are several different methods in `skimage` that can be used for edge detection, so we will examine only one in detail.

##Introduction to edge detection

To begin our introduction to edge detection, let us look at an image with a very simple edge – this grayscale image of two overlapped pieces of paper, one black and and one white:

<img src="https://datacarpentry.org/image-processing/fig/07-bw.jpg" alt="paper-edge" style="float: left; margin-right:10px;"/>

The obvious edge in the image is the vertical line between the black paper and the white paper. To our eyes, there is a quite sudden change between the black pixels and the white pixels. But, at a pixel-by-pixel level, is the transition really that sudden?

If we zoom in on the edge more closely, as in this image, we can see that the edge between the black and white areas of the image is not a clear-cut line.

<img src="https://datacarpentry.org/image-processing/fig/07-bw-edge-pixels.jpg" alt="paper edge pixels" style="float: left; margin-right:10px;"/>

We can learn more about the edge by examining the color values of some of the pixels. Imagine a short line segment, halfway down the image and straddling the edge between the black and white paper. This plot shows the pixel values (between 0 and 255, since this is a grayscale image) for forty pixels spanning the transition from black to white.

<img src="https://datacarpentry.org/image-processing/fig/07-bw-gradient.png" alt="color gradient plot" style="float: left; margin-right:10px;"/>

It is obvious that the "edge" here is not so sudden! So, any `skimage` method to detect edges in an image must be able to decide where the edge is, and place appropriately-colored pixels in that location.


##Canny edge detection

Our edge detection method in this workshop is *Canny edge detection*, created by John Canny in 1986. This method uses a series of steps, some incorporating other types of edge detection. The `skimage` `skimage.feature.canny()` function performs the following steps:

1. A Gaussian blur (that is characterized by the `sigma` parameter, see the previous lesson) is applied to remove noise from the image. (So if we are doing edge detection via this function, we should not perform our own blurring step.)

2. *Sobel edge detection* is performed on both the $x$ and $y$ dimensions, to find the intensity gradients of the edges in the image. Sobel edge detection computes the derivative of a curve fitting the gradient between light and dark areas in an image, and then finds the peak of the derivative, which is interpreted as the location of an edge pixel.

3. Pixels that would be highlighted, but seem too far from any edge, are removed. This is called *non-maximum suppression*, and the result is edge lines that are thinner than those produced by other methods.

4. A double threshold is applied to determine potential edges. Here extraneous pixels caused by noise or milder color variation than desired are eliminated. If a pixel's gradient value -- based on the Sobel differential -- is above the high threshold value, it is considered a strong candidate for an edge. If the gradient is below the low threshold value, it is turned off. If the gradient is in between, the pixel is considered a weak candidate for an edge pixel.

5. Final detection of edges is performed using *hysteresis*. Here, weak candidate pixels are examined, and if they are connected to strong candidate pixels, they are considered to be edge pixels; the remaining, non-connected weak candidates are turned off.

For a user of the `skimage.feature.canny()` edge detection function, there are three important parameters to pass in: `sigma` for the Gaussian filter in step one and the low and high threshold values used in step four of the process. These values generally are determined empirically, based on the contents of the image(s) to be processed.

The following code illustrates how the `skimage.feature.canny()` method can be used to detect the edges in an image. We will execute the program on this image:

<img src="https://datacarpentry.org/image-processing/fig/07-junk.jpg" alt="miscellaneous shapes" style="float: left; margin-right:10px;"/>

We are interested in finding the edges of the shapes in the image, and so the colors are not important. Our strategy will be to read the image as grayscale, and then apply Canny edge detection. Note that when reading the image with `skimage.io.imread(..., as_gray=True)` the image is converted to a `float64` grayscale with the original dtype range being mapped to values ranging from 0.0 to 1.0.


In [ ]:
# one-time imports and set up matplotlib
import skimage.io

import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 150

from matplotlib import pyplot as plt


In [ ]:
# read image, display for pedagogical purposes
image = skimage.io.imread('https://i.imgur.com/c1Y4NyB.jpg', as_gray=True)

skimage.io.imshow(image)
plt.show()

In [ ]:
# for convenience, set parameters in variables
sigma = 2.0
lowThresh = 0.1
highThresh = 0.3

In [ ]:
# perform Canny edge detection
import skimage.feature

edges = skimage.feature.canny(image, sigma, lowThresh, highThresh)

As we are using it here, the `skimage.feature.canny()` function takes four parameters. The first parameter is the input image. The `sigma` parameter determines the amount of Gaussian smoothing that is applied to the image. The next two parameters are the low and high threshold values for the fourth step of the process.

The result of this call is a binary image. In the image, the edges detected by the process are white, while everything else is black.

Displaying the edge image will let us see how well we did detecting the edges:

In [ ]:
# display the edge image
skimage.io.imshow(edges)
plt.show()

The colors in image that results are an artifact of the color map used by Matplotlib. We can get a better idea of how the edges really look by saving the `edges` image, downloading it, and viewing it via our operating system tools.

In [ ]:
# save edge image so it can be viewed in "true" colors
import skimage.util

edges = skimage.util.img_as_uint(edges)
skimage.io.imsave('edges.jpg', edges)

---
**Applying Canny edge detection to another image.**

Use Canny edge detection to produce a binary image of the color image at this URL (<a href="https://i.imgur.com/gpLmzNk.png">https://i.imgur.com/gpLmzNk.png</a>). Experiment with the sigma and threshold parameters until you arrive at an acceptable image. 

<img src="https://i.imgur.com/gpLmzNk.png" alt="Color maize roots" style="float: left; margin-right:10px;"/>

---